# Fixed-bed problem


Jialu Wang (jwang44@nd.edu) and Alex Dowling (adowling@nd.edu)

University of Notre Dame

This notebook conducts design of experiments for a fixed-bed model with the Pyomo.DOE.
    

In [1]:
from pyomo.environ import *
from pyomo.dae import *

import idaes

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
from itertools import permutations, product, combinations

### Import Pyomo.DOE

In [2]:
from fim_doe import *

### Import fixed-bed model

In [3]:
from fixed_bed_model import *

alpha option:
1 / (1 + exp(-x))
    Import ComponentSet from pyomo.common.collections.  (deprecated in 5.7.1)
    (called from <frozen importlib._bootstrap>:219)


### Set up fixed-bed model 

In [4]:
def add_model(doe, timepoints, time_start=0):
    '''
    add variables, equations and discretize the model time
    Arguments
    ---------
    timepoints: the timesteps
    time_start: where the timesteps start. For self defined timepoints it's 0. For experimental data it's -10.
    '''
    add_variables(doe, timesteps=timepoints, start=time_start)
    add_equations(doe)
    print ('the number of timepoints is', NFEt)
    
def discretize(doe, no_points=68, collo=False):
    '''
    Discretization 
    Arguments 
    ---------
    no_points: how many time invertals to divide. For self defined timepoints it's 69. For experimental data it's 68. 
    here it's 68
    collo: if using collocation or finite difference
    '''
    if collo:
        TransformationFactory('dae.collocation').apply_to(doe, nfe=no_points, ncp=3, scheme='LAGRANGE-RADAU', wrt=doe.t)
    else:
        TransformationFactory('dae.finite_difference').apply_to(doe, nfe=no_points, scheme='BACKWARD', wrt=doe.t)
        
    for i in doe.t: 
        doe.Q[i].fix()
    
    
def initialize(doe, init_point):
    '''
    Initialize the fixed bed 
    Arguments 
    ---------
    init_point: initial point, csv file
    '''
    print('The init point is', init_point)    
    store = pd.read_csv(init_point)
    position_max = store['position'].max()
    store['position'] = store['position'] / position_max

    initial_bed_csv(doe, store)
    fix_initial_bed(doe)
    
def generate_exp_addQ(feed, y, const_q):
    '''
    Generalize experiments adding Q
    Arguments 
    ---------
    feed: feed temperature, [K]
    yfeed: feed CO2 fraction, mol % 
    Q: extra heat added
    '''
    initialize_q = {}
    for t in exp_time_:
        initialize_q[t] = const_q
    dv_dict_overall = {'temp_feed': {0: feed}, 'yfeed': {0: y}, 'Q': initialize_q}
    return dv_dict_overall

def generate_exp(feed, y):
    '''
    Generalize experiments with no adding Q
    Arguments 
    ---------
    feed: feed temperature, [K]
    yfeed: feed CO2 fraction, mol % 
    '''
    dv_dict_overall = {'temp_feed': {0: feed}, 'yfeed': {0: y}}
    return dv_dict_overall

In [5]:
NFEt = 68 # No. of time points 
time_scale = 3200 # [s], time
collo = False # if not using collocation, finite difference is used 
exp_time_ = time_points(NFEt, time_scale) # The discretized time points 

scena_all = {'scena-name': [0], 'fitted_transport_coefficient':212, 'ua': np.log(5.0E6)}

# Starting point for the fixed-bed model 
initialize_point = '/media/psf/Home/CCSI-Toolset/fixed_bed_adsorption/fixed-bed-priors/2021oct8_f293_b293_y15_5e6.csv'


In [6]:
def model_integrate(scena, args=[True, False], exp_time=exp_time_, init_point=initialize_point):
    '''Work as the model creation function provided to Pyomo.DOE
    Note: When more than one Pyomo function are needed for creating, initializing, and discretizing
    models, they should be combined in this way and provide a ready-for-solving model name
    Arguments
    ---------
    args_list: Additional arguments for fixed-bed 
    exp_time: discretized time points 
    init_point: starting point for solving the model 
    
    Return 
    ------
    test: model for Pyomo.DOE
    '''
    test = create_model(scena, temp_feed=313.15, temp_bath=293.15, y=0.15, doe_model=args[0], k_aug=args[1], opt=False, diff=0)
    add_model(test, exp_time)
    discretize(test)
    initialize(test, init_point)
    return test


### Specify inputs for Pyomo.DOE

In [7]:
# specify parameters 
parameter_dict = {'fitted_transport_coefficient':212, 'ua': np.log(5.0E6)}

# specify design variables 
#dv_pass = {'temp_feed': None, 'yfeed': None, 'Q':exp_time_}
dv_pass = {'temp_feed':None, 'yfeed': None}
#if_optimization = {'temp_feed': True, 'yfeed': True, 'Q':False}

# specify measurements 
t_measure = exp_time_
measurement_pass = {'FCO2': {19: t_measure}, 'temp': {10: t_measure, 19:t_measure}}
var = {'FCO2': {19: 0.00001}, 'temp': {10: 1, 19:1}}

measure_class = Measurements(measurement_pass, variance=var)

All measurements are flattened.
Flatten measurement name: ['FCO2_index_19', 'temp_index_10', 'temp_index_19']


In [8]:
# empty prior
prior_all = np.zeros((2,2))

prior_pass=np.asarray(prior_all)

#L_initials = np.linalg.cholesky(prior_pass)
#print(L_initials)

print('The prior information FIM:', prior_pass)
print('Prior Det:', np.linalg.det(prior_pass))
print('Eigenvalue of the prior experiments FIM:', np.linalg.eigvals(prior_pass))
print('Eigenvalue of the prior experiments FIM:', np.linalg.eigh(prior_pass)[1])

The prior information FIM: [[0. 0.]
 [0. 0.]]
Prior Det: 0.0
Eigenvalue of the prior experiments FIM: [0. 0.]
Eigenvalue of the prior experiments FIM: [[1. 0.]
 [0. 1.]]


## Compute FIM
Compute a square MBDoE problem for a specified experiment

In [ ]:
# choose from 'sequential_finite', 'direct_kaug'
sensi_opt = 'sequential_finite'
#sensi_opt = 'direct_kaug'

# specify model for different modes
if (sensi_opt == 'sequential_finite'):
    args_ = [True, False]
    
elif (sensi_opt =='direct_kaug'):
    args_ = [True, True]
    
# define an experiment
#exp1 = generate_exp(293.15, 0.15, 0)
exp1 = generate_exp(293.15, 0.15)

In [ ]:
doe_object = DesignOfExperiments(parameter_dict, dv_pass,
                                 measure_class, model_integrate,
                                prior_FIM=prior_pass, discretize_model=None, args=args_)

# Read the solution stored
read_name = '/media/psf/Home/CCSI-Toolset/fixed_bed_adsorption/fixed-bed-priors/20220205_293_293_40_5e6' 

# use .compute_FIM() function
result = doe_object.compute_FIM(exp1, mode=sensi_opt, FIM_store_name = 'fixed_bed.csv', 
                                store_output = None, read_output=read_name,
                                extract_single_model = extract3_v2,
                                formula='central')

# analyze the result object
result.calculate_FIM(doe_object.design_values)
    
    

In [ ]:
print('======Result summary======')
print('Four design criteria log10() value:')
print('A-optimality:', np.log10(result.trace))
print('D-optimality:', np.log10(result.det))
print('E-optimality:', np.log10(result.min_eig))
print('Modified E-optimality:', np.log10(result.cond))

In [ ]:
# Addiontional: to store the solution for one specific model 
#dataframe = extract3_v2(doe_object.models[0])
#dataframe.to_csv('test6.csv')

# Additional: to show the breakthrough curve for adsorption 
#breakthrough_modify2(doe_object.models[0])

## Gradient-based optimization 

In [ ]:
# define a starting point for optimization 
exp1 = generate_exp(293.15, 0.15)

### Sequential initial solution 

In [ ]:
fim_init = result.FIM
print(fim_init)
    
l_init = np.linalg.cholesky(fim_init)
print(l_init)

In [ ]:
args_ = [True, False]

doe_object = DesignOfExperiments(parameter_dict, dv_pass,
                                 measure_class, model_integrate,
                                prior_FIM=prior_pass, discretize_model=None, args=args_)

square_result, optimize_result = doe_object.optimize_doe(exp1, if_optimize=True, if_Cholesky=True,
                                                         scale_nominal_param_value=True, objective_option='det',
                                                         L_initial=l_init, fim_initial=fim_init)


In [ ]:
print('======Result summary======')
print('This optimization is solved with status:', optimize_result.status)
print('It gives solution:', optimize_result.solution)
print('The log10(OF) optimized is:', optimize_result.obj_value)
print('The result FIM is:', optimize_result.FIM)

t_list = []
for t in optimize_result.model.t:
    t_list.append(t)

T_list = []
for i in t_list:
    T_list.append(value(optimize_result.model.T[i]))

si=16
plt.rc('axes', titlesize=si)
plt.rc('axes', labelsize=si)
plt.rc('xtick', labelsize=si)
plt.rc('ytick', labelsize=si)
plt.rc('legend', fontsize=12)
    
plt.plot(t_list, T_list, 'b', linewidth=2)
#plt.scatter(t_list, T_list, 'b')
plt.ylabel('T [$K$]')
plt.xlabel('Time [$h$]')
plt.show()

## Exploratory analysis 

### Define enumeration ranges

In [ ]:
# Define design ranges 
dv_apply_name = ['temp_feed','yfeed']
# go over these designs
design_ranges = [list(np.linspace(293,353,13)), list(np.linspace(0.1,0.4,13))]
# control time points 
dv_apply_time = [[0],[0]]

# Define experiments
exp1 = generate_exp(313.15, 293.15, 0.15, 0)
    
# choose from 'simultaneous', 'sequential', 'sipopt'
sensi_opt = 'sequential_finite'
#sensi_opt = 'direct_kaug'

### Enumeration

In [ ]:
doe_object = DesignOfExperiments(parameter_dict, dv_pass,
                                 measure_class, createmod,
                                prior_FIM=prior_pass, discretize_model=None, args=args_)

all_fim = doe_object.run_grid_search(exp1, design_ranges,dv_apply_name, dv_apply_time, 
                                     mode=sensi_opt, tee_option=True,
                                    scale_nominal_param_value=True, 
                                     store_name='20211031_heatmap'
                                    )

### Draw heatmaps

In [ ]:
fixed = {}
all_fim.figure_drawing(fixed, ['temp_feed','y'], 'fixed bed', '$T_{feed}$ [K]', 'y [mol%]' )